# Hive
![Hive](https://hive.apache.org/images/hive_logo_medium.jpg)

- https://hive.apache.org/

## Setup

- version 3.1.2

In [ ]:
%%bash

# Download package
cd /opt/pkgs
wget -q -c https://downloads.apache.org/hive/hive-3.1.2/apache-hive-3.1.2-bin.tar.gz

# unpack file and create link
tar -zxf apache-hive-3.1.2-bin.tar.gz -C /opt
ln -s /opt/apache-hive-3.1.2-bin /opt/hive

# update envvars.sh
cat >> /opt/envvars.sh << EOF
# Hive
export HIVE_HOME=/opt/hive
export PATH=\${PATH}:\${HIVE_HOME}/bin

EOF

# Fix guava and slf4j versions
rm /opt/hive/lib/guava-19.0.jar
cp /opt/hadoop/share/hadoop/hdfs/lib/guava-27.0-jre.jar /opt/hive/lib
rm /opt/hive/lib/log4j-slf4j-impl-2.10.0.jar

cat /opt/envvars.sh

In [ ]:
# Load environment variables
%load_ext dotenv
%dotenv -o /opt/envvars.sh
%env

## Hadoop configuration (for beeline)

- core-site.xml

```xml
<configuration>
...
<property>
  <name>hadoop.proxyuser.hadoop.groups</name>
  <value>*</value>
</property>
<property>
  <name>hadoop.proxyuser.hadoop.hosts</name>
  <value>*</value>
</property>
</configuration>
```

## Hive Metastore

- using local Derby database

### Create directory in HDFS

In [ ]:
%%bash

hdfs dfs -mkdir -p /user/hive/warehouse
hdfs dfs -chmod g+w /user/hive/warehouse

### Initialize database

In [ ]:
%%bash

mkdir $HIVE_HOME/hiveserver2
cd $HIVE_HOME/hiveserver2
$HIVE_HOME/bin/schematool -dbType derby -initSchema 2> /dev/null

### Start hiveserver2

In [ ]:
%%bash

cd /opt/hive/hiveserver2
nohup /opt/hive/bin/hive --service hiveserver2 \
--hiveconf hive.security.authorization.createtable.owner.grants=ALL \
--hiveconf hive.root.logger=INFO,console > hiveserver2.out 2>&1 &
echo $! > hiveserver2.pid

## Example

- SF Bay Area Bike Share (https://www.kaggle.com/benhamner/sf-bay-area-bike-share)
- stations.csv and trips.csv

In [ ]:
%%bash

mkdir /opt/datasets/bikeshare
tar -zxf hivedataset.tgz -C /opt/datasets/bikeshare

cd /opt/datasets/bikeshare
ls

hdfs dfs -mkdir -p bikeshare/stations
hdfs dfs -put stations.csv bikeshare/stations
hdfs dfs -mkdir -p bikeshare/trips
hdfs dfs -put trips.csv bikeshare/trips

## Connect with beeline

1. Run beeline

```bash
source /opt/envvars.sh
beeline -n hadoop -u jdbc:hive2://localhost:10000
```

2. Configure jobs executor

```sql
SET hive.execution.engine=mr;
SET mapreduce.framework.name=yarn;
```

3. Create bikeshare database

```sql
CREATE DATABASE bikeshare;
SHOW DATABASES;
USE bikeshare;
```

4. Create stations table

```sql
CREATE EXTERNAL TABLE stations (
station_id INT,
name STRING,
lat DOUBLE,
long DOUBLE,
dockcount INT,
landmark STRING,
installation STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/stations';
```

5. Create trips table

```sql
CREATE EXTERNAL TABLE trips (
trip_id INT,
duration INT,
start_date STRING,
start_station STRING,
start_terminal INT,
end_date STRING,
end_station STRING,
end_terminal INT,
bike_num INT,
subscription_type STRING,
zip_code STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/trips';
```

6. Show tables

```sql
SHOW TABLES;
DESCRIBE stations;
DESCRIBE trips;
DESCRIBE FORMATTED stations;
DESCRIBE FORMATTED trips;
```

7. Run query - number of trips per terminal

```sql
SELECT start_terminal, start_station, COUNT(1) AS count
FROM trips
GROUP BY start_terminal, start_station
ORDER BY count
DESC LIMIT 10;
```

8. Run query - join between stations and trips

```sql
SELECT t.trip_id, t.duration, t.start_date, s.name, s.lat, s.long, s.landmark
FROM stations s
JOIN trips t ON s.station_id = t.start_terminal
LIMIT 10;
```

9. Exit beeline

```sql
!quit
```

In [ ]:
%%bash

cd /opt/hive/hiveserver2

# kill hiveserver2
kill $(cat hiveserver2.pid)
rm hiveserver2.pid